In [279]:
import tkinter as tk
from PIL import ImageTk, Image
import numpy as np
import colour as color

In [280]:
# Represents a 3D Vector in space with a color
class Vector:
    def __init__(self, x, y, z, color = color.Color("red")):
        self.x = x
        self.y = y
        self.z = z
        self.color = color

        # Matrix representation of the Vector
        self.matrix = np.array([x, y, z])

    # Adds two vectors
    def __add__(self, vector):
        return Vector(self.x + vector.x, self.y + vector.y, self.z + vector.z, vector.color)

    # Substracts two vectors
    def __sub__(self, vector):
        return Vector(self.x - vector.x, self.y - vector.y, self.z - vector.z, vector.color)

    # Displays the Vector information as a string when printed
    def __str__(self):
        return f"Vector({self.x}, {self.y}, {self.z}, {self.color})"

    def magnitude(self):
        return (self.x ** 2 + self.y ** 2 + self.z ** 2) ** 0.5

    # Returns the Vector with the same direction b a magnitude of 1
    def normalized(self):
        return Vector(self.x / self.magnitude(), self.y / self.magnitude(), self.z / self.magnitude(), self.color)

    # Returns the vector perpendicular to this and another vector
    def cross_product(self, vector):
        return Vector(self.y * vector.z - self.z * vector.y,
                    self.z * vector.x - self.x * vector.z,
                    self.x * vector.y - self.y * vector.x, 0)
    
    def dot_product(self, vector):
        return np.dot(self.np_matrix(), vector.np_matrix())
    
    def norm_product(self, vector):
        return np.linalg.norm(self.np_matrix() * np.linalg.norm(vector.np_matrix()))

    def np_matrix(self):
        return np.array([self.x, self.y, self.z])
    
    # Returns the angle between this vector and another
    def angle(self, vector):
        return np.arccos(self.dot_product(vector) / self.norm_product(vector))
    
    # Draws the point on screen
    def draw_vertex(self, screen):
        screen.draw_Vector(self, 5)

In [281]:
a = Vector(1, 0, 0)
b = Vector(0, 1, 0)

In [282]:
# Represents a 3D polygon in space with 3 vertices and a base color
class Polygon:
    def __init__(self, vertices, normal, color):
        self.vertices = vertices
        self.normal = normal
        self.color = color

    # Displays the Polygon information as a string when printed
    def __str__(self):
        return f"Polygon({self.vertices[0]}, {self.vertices[1]}, {self.vertices[2]}, {self.color})"
    
    # Returns a Vertex list of the points along the line between two vertices
    def _points_along_line(self, vertex1, vertex2, steps):
        points = []
        for i in range(steps + 1):
            x = vertex1.x + (vertex2.x - vertex1.x) * i / steps
            y = vertex1.y + (vertex2.y - vertex1.y) * i / steps
            z = vertex1.z + (vertex2.z - vertex1.z) * i / steps
            points.append(Vector(x, y, z, self.color))
        return points
    
    def draw_line(self, screen, vertex1, vertex2, steps):
        print("Drawing line")
        points = self._points_along_line(vertex1, vertex2, steps)
        lights = screen.light_model
        observer = screen.camera.origin

        for point in points:
            print(point)
            self.draw_lit(screen, point, lights, observer)

    def _light_direction(self, vertex, lights):
        light_direction = (lights.point_lights[0] - vertex).normalized()
        light_direction.color = color.Color("green")
        return light_direction
    
    def _observer_direction(self, vertex, observer):
        obersver_direction = (observer - vertex).normalized()
        obersver_direction.color = color.Color("purple")
        return obersver_direction
    
    def _reflection_direction(self, vertex, lights, observer):
        light_direction = self._light_direction(vertex, lights)
        reflection_direction = 2 * np.dot(self.normal.np_matrix(), light_direction.np_matrix()) * self.normal.np_matrix() - light_direction.np_matrix()
        reflection_direction = Vector(reflection_direction[0], reflection_direction[1], reflection_direction[2])
        reflection_direction.color = color.Color("red")
        return reflection_direction
    
    def _ambient_light(self, lights):
        return lights.ambient_coefficient

    def _diffuse_light(self, vertex, lights):
        light_direction = self._light_direction(vertex, lights)
        return lights.diffuse_coefficient * max(0, light_direction.dot_product(self.normal))
    
    def _specular_light(self, vertex, lights, observer):
        reflection_direction = self._reflection_direction(vertex, lights, observer)
        observer_direction = self._observer_direction(vertex, observer)
        return lights.specular_coefficient * np.dot(reflection_direction.np_matrix(), observer_direction.np_matrix()) ** lights.specular_exponent

    def draw_vertices(self, screen):
        for vertex in self.vertices:
            vertex.draw_vertex(screen)

    def draw_vertices_normals(self, screen):
        for vertex in self.vertices:
            (vertex + self.normal).draw_vertex(screen)

    def draw_vertices_light_direction(self, screen, lights):
        for vertex in self.vertices:
            self._light_direction(vertex, lights).draw_vertex(screen)

    def draw_vertices_observer_direction(self, screen, observer):
        for vertex in self.vertices:
            self._observer_direction(vertex, observer).draw_vertex(screen)

    def draw_vertices_reflection_direction(self, screen, lights, observer):
        for vertex in self.vertices:
            self._reflection_direction(vertex, lights, observer).draw_vertex(screen)

    def draw_lit(self, screen, vertex, lights, observer):
        ambient = self._ambient_light(lights)
        diffuse = self._diffuse_light(vertex, lights)
        specular = self._specular_light(vertex, lights, observer)
        luminance = ambient + diffuse + specular

        vertex.color.luminance = luminance

        if vertex.color.luminance > 1:
            vertex.color.luminance = 1
            vertex.color = color.Color("white")

        # vertex.color.luminance = luminance
        screen.draw_Vector(vertex, 3)

    def draw_lit_vertices(self, screen, lights, observer):
        for vertex in self.vertices:
            self.draw_lit(screen, vertex, lights, observer)

    def draw_lit_borders(self, screen, lights, observer):
        self.draw_line(screen, self.vertices[0], self.vertices[1], 20)
        self.draw_line(screen, self.vertices[1], self.vertices[2], 20)
        self.draw_line(screen, self.vertices[2], self.vertices[0], 20)

    def draw_lit_faces(self, screen):
        lights = screen.light_model
        observer = screen.camera.origin
        origin = self.vertices[0]
        destinations = self._points_along_line(self.vertices[1], self.vertices[2], 50)
        print(destinations)

        for destination in destinations:
            self.draw_line(screen, origin, destination, 50)

In [283]:
# Represents the properties and transformation matrices of a camera
class Camera:
    def __init__(self, u, v, w, origin, height, width, focus, far_plane, close_plane):
        self.u = u
        self.v = v
        self.w = w
        self.origin = origin
        self.height = height
        self.width = width
        self.focus = focus
        self.far_plane = far_plane
        self.close_plane = close_plane

        # Aspect ratio of the camera
        self.aspect_ratio = width / height

        # Matrix representation of the camera
        self.camera_matrix = np.array([
            [self.u.x, self.v.x, self.w.x],
            [self.u.y, self.v.y, self.w.y],
            [self.u.z, self.v.z, self.w.z]
        ])

        # Transposed matrix representation of the camera
        self.transposed_camera_matrix = np.transpose(self.camera_matrix)

        # Matrix representation of the origin
        self.origin_matrix = -np.matmul(self.transposed_camera_matrix, self.origin.matrix)

        # Transformation matrix of the camera
        self.transformation_matrix = np.array([
            [self.u.x, self.u.y, self.u.z, self.origin_matrix[0]],
            [self.v.x, self.v.y, self.v.z, self.origin_matrix[1]],
            [self.w.x, self.w.y, self.w.z, self.origin_matrix[2]],
            [0, 0, 0, 1]
        ])

        # Projection matrix of the camera
        self.projection_matrix = np.array([
            [self.focus / self.width, 0, 0, 0],
            [0, self.focus / self.height, 0, 0],
            [0, 0, -(self.far_plane + self.close_plane) / (self.far_plane - self.close_plane), -2 * self.far_plane * self.close_plane / (self.far_plane - self.close_plane)],
            [0, 0, -1, 0]
        ]) 

In [284]:
# Represents screen space and displays the objects on it
class Screen:
    def __init__(self, width, height, window, camera, light_model):
        self.width = width
        self.height = height
        self.window = window
        self.camera = camera
        self.light_model = light_model

        # Calculates the delta between the center of the screen and the center of the window
        self.x_delta = (width - 1) / (camera.width * 2)
        self.y_delta = (height - 1) / (camera.height * 2)

        # Sets up the window
        self.window.geometry(str(width) + "x" + str(height))
        self.canvas = tk.Canvas(self.window, width = width, height = height)    
        self.canvas.configure(background = "black")
        self.canvas.pack()

        # Displays lights
        for light in self.light_model.point_lights:
            self.draw_Vector(light, 5)


    # PRIVATE VARIABLES
    # Transforms a Vector in world space (x, y, z) to projection space (x, y, z, w)
    def _world_to_projection(self, Vector):
        Vector_4d = np.append(Vector.matrix, 1)
        transformed = np.matmul(self.camera.transformation_matrix, Vector_4d)
        projected = np.matmul(self.camera.projection_matrix, transformed)
        projected_scaled = projected / projected.max()
        return projected_scaled
    
    # Transforms a Vector in world space (x, y, z) to screen coordinates (x, y)
    def _world_to_screen_coordinate(self, Vector):
        Vector_2d = self._world_to_projection(Vector)
        display_x = (self.x_delta * Vector_2d[0]) + (self.x_delta * self.camera.width) + self.camera.width
        display_y = (self.y_delta * Vector_2d[1]) + (self.y_delta * self.camera.height) + self.camera.height
        return [round(display_x), round(display_y)]
    
    # Calculates the steps between two coordinates in screen space (x, y) and the delta in each direction
    def _calculate_pixel_steps(self, a, b):
        # Calculates the difference between the Vectors
        delta_x = a[0] - b[0]
        delta_y = a[1] - b[1]

        # Calculates the number of steps needed to draw the line on the larger axis and scales the delta accordingly
        if abs(delta_x) > abs(delta_y):
            steps = abs(delta_x)
            delta_y = delta_y / steps
            delta_x = delta_x / steps
        else:
            steps = abs(delta_y)
            delta_y = delta_y / steps
            delta_x = delta_x / steps

        return steps, -delta_x, -delta_y
    

    # PUBLIC VARIABLES
    # Draws a circle on the screen with the given radius and color from a Vector in world space (x, y, z)
    def draw_Vector(self, Vector, size = 1):
        screen_coordinate = self._world_to_screen_coordinate(Vector)
        self.canvas.create_oval(screen_coordinate[0] - size, screen_coordinate[1] - size, screen_coordinate[0] + size, screen_coordinate[1] + size, fill = Vector.color, outline = Vector.color)

    # Draws a pixel on a screen coordinate (x, y) with the given color
    def draw_coordinate(self, x, y, color = "white"):
        self.canvas.create_oval(x, y, x, y, fill = color, outline = color)

    # Creates a list of screen coordinates (x, y) between two Vectors in world space (x, y, z)
    def world_line_coordinates(self, a, b):
        # Converts Vectors from world space to screen space
        coordinate_a = self._world_to_screen_coordinate(a)
        coordinate_b = self._world_to_screen_coordinate(b)

        # Calculates the number of steps and delta in each direction
        steps, delta_x, delta_y = self._calculate_pixel_steps(coordinate_a, coordinate_b)

        # Creates the list of coordinates
        line_coordinates = []
        for i in range(steps + 1):
            line_coordinates.append((round(coordinate_a[0] + i * delta_x), round(coordinate_a[1] + i * delta_y)))

        return line_coordinates
    
    # Draws a line in the screen between two Vectors in world space (x, y, z)
    def draw_line_between_Vectors(self, a, b):
        coordinates = self.world_line_coordinates(a, b)
        for i in range(len(coordinates)):
            self.draw_coordinate(coordinates[i][0], coordinates[i][1])

In [285]:
# Represents the illumination setup of the scene
class LightModel:
    def __init__(self, ambient_coefficient, diffuse_coefficient, specular_coefficient, specular_exponent):
        self.ambient_coefficient = ambient_coefficient
        self.diffuse_coefficient = diffuse_coefficient
        self.specular_coefficient = specular_coefficient
        self.specular_exponent = specular_exponent
        self.point_lights = []

    # Adds a point light to the scene as a Vector in world space (x, y, z)
    def add_point_light(self, Vector):
        self.point_lights.append(Vector)

In [286]:
# Represents a 3D object in space
class Cube:
    def __init__(self, size):
        self.size = size
        self.vertices = self._calculate_vertices()
        self.polygons = self.create_polygons()

    # Calculates the vertices of the cube
    def _calculate_vertices(self):
        vertices = []
        offset = self.size - self.size/2
        for x in range(2):
            for y in range(2):
                for z in range(2):
                    vertices.append(Vector(x * self.size - offset, y * self.size - offset, z * self.size - offset))
        return vertices
    
    # Creates the polygons of the cube
    def create_polygons(self):
        return {
            "back_1": Polygon([self.vertices[0], self.vertices[2], self.vertices[6]], Vector(0, 0, -1), color.Color("green")),
            "back_2": Polygon([self.vertices[6], self.vertices[4], self.vertices[0]], Vector(0, 0, -1), color.Color("yellow")),
            "left_1": Polygon([self.vertices[2], self.vertices[3], self.vertices[1]], Vector(-1, 0, 0), color.Color("orange")),
            "left_2": Polygon([self.vertices[2], self.vertices[1], self.vertices[0]], Vector(-1, 0, 0), color.Color("purple")),
            "right_1": Polygon([self.vertices[7], self.vertices[4], self.vertices[5]], Vector(1, 0, 0), color.Color("pink")),
            "right_2": Polygon([self.vertices[7], self.vertices[4], self.vertices[6]], Vector(1, 0, 0), color.Color("cyan")),
            "top_1": Polygon([self.vertices[2], self.vertices[3], self.vertices[7]], Vector(0, 1, 0), color.Color("brown")),
            "top_2": Polygon([self.vertices[2], self.vertices[7], self.vertices[6]], Vector(0, 1, 0), color.Color("lime")),
            "bottom_1": Polygon([self.vertices[1], self.vertices[0], self.vertices[4]], Vector(0, -1, 0), color.Color("gray")),
            "bottom_2": Polygon([self.vertices[1], self.vertices[4], self.vertices[5]], Vector(0, -1, 0), color.Color("wheat")),
            "front_1": Polygon([self.vertices[0], self.vertices[3], self.vertices[5]], Vector(0, 0, 1), color.Color("red")),
            "front_2": Polygon([self.vertices[3], self.vertices[7], self.vertices[5]], Vector(0, 0, 1), color.Color("blue")),
        }
    
    # Draws the vertices of the cube on screen
    def draw_vertices(self, screen):
        for vertex in self.vertices:
            screen.draw_Vector(vertex, 5)
    
    # Draws the wireframe of the cube on screen
    def draw_wireframe(self, screen):
        for polygon in self.polygons.values():
            polygon.draw_lit_borders(screen, screen.light_model, screen.camera.origin)

    def draw_faces(self, screen):
        for polygon in self.polygons.values():
            polygon.draw_lit_faces(screen)

In [287]:
# Create the main window
window = tk.Tk()
window.title("Phong Illumination Model")

''

In [288]:
# Camera declaration
u = Vector(-1, 0, 0)
v = Vector(0, -1, 0)
w = Vector(0, 0, -1)
origin = Vector(0, 0, -5)
cam = Camera(u, v, w, origin, 1, 1, 1, 12, 3)

# Light declaration
lights = LightModel(0.5, 0.5, 0.5, 1)
lights.add_point_light(Vector(4, 4, 3, color.Color("white")))

# Scren declaration
screen = Screen(1000, 1000, window, cam, lights)

# Cube delcaration
cube = Cube(2)
cube.draw_wireframe(screen)
cube.draw_faces(screen)

# a = Polygon([Vector(0, 0, 0), Vector(2, -2, 2), Vector(4, 0, 0)], Vector(0, 1, 0), color.Color("red"))
# a.draw_lit_vertices(screen, lights, cam.origin)






Drawing line
Vector(-1.0, -1.0, -1.0, green)
Vector(-1.0, -0.9, -1.0, #003f00)
Vector(-1.0, -0.8, -1.0, #004100)
Vector(-1.0, -0.7, -1.0, #004300)
Vector(-1.0, -0.6, -1.0, #004500)
Vector(-1.0, -0.5, -1.0, #004700)
Vector(-1.0, -0.4, -1.0, #004900)
Vector(-1.0, -0.30000000000000004, -1.0, #004b00)
Vector(-1.0, -0.19999999999999996, -1.0, #004d00)
Vector(-1.0, -0.09999999999999998, -1.0, #004f00)
Vector(-1.0, 0.0, -1.0, #005100)
Vector(-1.0, 0.10000000000000009, -1.0, #005300)
Vector(-1.0, 0.19999999999999996, -1.0, #050)
Vector(-1.0, 0.30000000000000004, -1.0, #005700)
Vector(-1.0, 0.3999999999999999, -1.0, #005900)
Vector(-1.0, 0.5, -1.0, #005b00)
Vector(-1.0, 0.6000000000000001, -1.0, #005d00)
Vector(-1.0, 0.7, -1.0, #005f00)
Vector(-1.0, 0.8, -1.0, #006100)
Vector(-1.0, 0.8999999999999999, -1.0, #006300)
Vector(-1.0, 1.0, -1.0, DarkGreen)
Drawing line
Vector(-1.0, 1.0, -1.0, #060)
Vector(-0.9, 1.0, -1.0, #060)
Vector(-0.8, 1.0, -1.0, #006900)
Vector(-0.7, 1.0, -1.0, #006b00)
Vector(

Vector(0.72, 0.0, -1.0, #7d7d00)
Vector(0.7087999999999999, -0.040000000000000036, -1.0, #7c7c00)
Vector(0.6976, -0.08000000000000007, -1.0, #7b7b00)
Vector(0.6863999999999999, -0.1200000000000001, -1.0, #7a7a00)
Vector(0.6752, -0.15999999999999992, -1.0, #787800)
Vector(0.6639999999999999, -0.19999999999999996, -1.0, #770)
Vector(0.6527999999999999, -0.24, -1.0, #767600)
Vector(0.6416, -0.28, -1.0, #747400)
Vector(0.6304000000000001, -0.32000000000000006, -1.0, #737300)
Vector(0.6192, -0.3600000000000001, -1.0, #727200)
Vector(0.608, -0.3999999999999999, -1.0, #707000)
Vector(0.5968, -0.43999999999999995, -1.0, #6f6f00)
Vector(0.5855999999999999, -0.48, -1.0, #6e6e00)
Vector(0.5744, -0.52, -1.0, #6d6d00)
Vector(0.5631999999999999, -0.56, -1.0, #6b6b00)
Vector(0.5519999999999999, -0.6000000000000001, -1.0, #6a6a00)
Vector(0.5408, -0.6399999999999999, -1.0, #696900)
Vector(0.5296, -0.6799999999999999, -1.0, #676700)
Vector(0.5184, -0.72, -1.0, #660)
Vector(0.5072, -0.76, -1.0, #656500)


In [289]:
# Tkinter loop
window.mainloop()

: 